In [ ]:
# US states outlines
# https://stackoverflow.com/questions/1814169/geographical-boundaries-of-states-provinces-google-maps-polygon
xml_file = "states.xml"
import xml.etree.ElementTree as ET
tree = ET.parse(xml_file)
root = tree.getroot()
name_to_boundary = {}
allpoints = []
for state in root:
    name = state.attrib["name"]
    #print name
    boundary = []
    for point in state:
        lat = float(point.attrib["lat"])
        lng = float(point.attrib["lng"])
        #pt = [lat, lng]
        pt = [lng, lat]
        boundary.append(pt)
        allpoints.append(pt)
    name_to_boundary[name] = boundary
    
from jp_doodle import dual_canvas
from IPython.display import display
states = dual_canvas.DualCanvasWidget(width=520, height=320)
states.check_jquery()

states.text(text="Longitude", y=-60, x=250, align="center", font="bold 20px Arial",)
states.text(text="Latitude", y=150, x=-50, align="center", degrees=90, font="bold 20px Arial",)
minlng = min(x[0] for x in allpoints)
maxlng = max(x[0] for x in allpoints)
minlat = min(x[1] for x in allpoints)
maxlat = max(x[1] for x in allpoints)
display(states)

# image underlay
earth_image = 'Earthmap1000x500.jpg'
def latitude_pixel(lat):
    "pixel from top edge = (90 - latitude) / 0,36"
    return (90.0 - lat) / 0.36
def longitude_pixel(lng):
    "pixel from left hand side = (180 + longitude) / 0,36"
    return (180 + lng) / 0.36
sx = longitude_pixel(minlng)
sy = latitude_pixel(maxlat)
sWidth = (maxlng - minlng) / 0.36
sHeight = (maxlat - minlat) / 0.36
print "sx, sy, sWidth, sHeight", sx, sy, sWidth, sHeight
states.name_image_url("earth", earth_image)
states.named_image("earth", 0,0,500,300, degrees=0, 
                   sx=sx, sy=sy, sWidth=sWidth, sHeight=sHeight)

frame = states.named_frame_region("latlong", 0,0,500,300,minlng, minlat, maxlng, maxlat)
for name in name_to_boundary:
    points = name_to_boundary[name]
    # add invisible filled polygon for state interior
    frame.polygon(points=points, name=name, color="rgba(100,200,0,0.5)")
    # add unnamed state border
    frame.polygon(points=points, color="#38f", fill=False)
frame.lower_left_axes(minlng, minlat, maxlng, maxlat)
#print minlng, minlat, maxlng, maxlat
states.fit()

states.js_init("""
var info = $("<div>info here</div>").appendTo(element);
var dialog = $("<div>dialog text</div>").appendTo(element);
dialog.dialog();
dialog.dialog("close");

element.text
var last_name = null
count = 0;
var mouse_move = function(event) {
    count += 1;
    var name = event.canvas_name;
    if ((last_name) && (last_name!=name)) {
        // obscure the interior)
        element.change_element(last_name, {color: "rgba(100,200,0,0.5)"})
        last_name = null;
        dialog.dialog("close");
    }
    if (name) {
        // make the interior transparent
        element.change_element(name, {color: "rgba(0,0,0,0)"})
        last_name = name;
        var pos = { my: "left+10 top+10", at: "left bottom", of: event }
        dialog.dialog("option", "position", pos);
        dialog.html("<div>"+name+"</div>");
        dialog.dialog("open");
    }
    info.html("<div>name="+name+"; last_name="+last_name
      +"; count="+count+"</div>");
}
element.on_canvas_event("mousemove", mouse_move);
""")

In [ ]:
xxx = dual_canvas.DualCanvasWidget(width=520, height=320)
display(xxx)
#minlng, minlat, maxlng, maxlat = 10, 10, 100, 100
#xxx.lower_left_axes(minlng, minlat, maxlng, maxlat)
#xxx.fit()
xxx.js_init("""
element.lower_left_axes({
    min_x:minlng, min_y:minlat, max_x:maxlng, max_y:maxlat
})
""", minlng=minlng, maxlng=maxlng, minlat=minlat, maxlat=maxlat)

In [ ]:
# https://commons.wikimedia.org/wiki/File:Earthmap1000x500.jpg
#pixel from top edge = (90 - latitude) / 0,36
#pixel from left hand side = (180 + longitude) / 0,36